# <font color=#5b797e> Figure 3 | Spinal cord func / morphometry coupling </font>
<hr style="border:1px solid black">

to read: https://elifesciences.org/articles/62116

### Imports

In [ ]:
import sys,json
import glob, os
import pandas as pd
import numpy as np
from palettable.colorbrewer.sequential import GnBu_9
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

#statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import pearsonr
from scipy.stats import spearmanr

main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project"
sys.path.append(main_dir + "/code/")
from brsc_statistics import Statistics 

from plotting import Plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load config file ------------------------------------------------------------
config_file=main_dir + '/config/analyses/brsc_sfc.json' # the notebook should be in 'xx/notebook/' folder #config_proprio
with open(config_file) as config_f: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_f)
    
plot=Plotting(config_file,"test")
stat_func=Statistics(config=config,ana_dir="",analysis="")

IDs=config["participants_IDs_ALL"]
structure="spinalcord"
output_dir= config["project_dir"] +"/figures/f02_functional_sc/fc_tps_coupling/figures"
metadata = pd.read_csv(config["project_dir"] + config["population_infos"], delimiter='\t')


<hr style="border:1px solid black">

### <font color=#5b797e> A. Load data
#### <font color=#5b797e> A.1 Matrices (TPS and FC)

In [ ]:
files={};matrix={};df_matrix_all={};age=[];sex=[]

#---- Initiate variables
for modality in ["func_tp","func_fc"]:
    #----- Load indiv data
    files[modality]=[];matrix[modality]=[]
    for ID_nb, ID in enumerate(IDs):
        files[modality].append(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["first_level"]["global"].format(ID))[0])
        matrix[modality].append(np.array(pd.read_csv(files[modality][ID_nb],header=None)))

        if modality=="func_tp":
            age.append(metadata[metadata["participant_id"] == ID]["age"].values[0])
            sex.append(metadata[metadata["participant_id"] == ID]["sex"].values[0])
    matrix[modality] =np.array( matrix[modality]) 
    
    #---- Load group data aÀnd filter the dataframe
    df_matrix_all[modality]=pd.read_csv(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["second_level_matrix"])[0])     

# ----- Grouped all matrix in the same DF
all_df=df_matrix_all["func_tp"];all_df["fc"]=df_matrix_all["func_fc"]["fcorr"];
all_df["rois"] = all_df["seed1"] + "_" + all_df["seed2"]

#### <font color=#5b797e> A.2 Load features values (temporal profile and fc)

In [ ]:
df_metrics_all={};df_metrics_reduced={};
for modality in ["func_tp","func_fc",]:
    
    #---- Load group data and filter the dataframe
    df_metrics_all[modality]=pd.read_csv(glob.glob(config["project_dir"] + config[modality]["analysis_dir"]["spinalcord"] + config[modality]["second_level_metrics"])[0])
        
    if modality=="func_tp":
        functional_features = df_metrics_all[modality].columns[np.r_[8:11,12:19,20:33]] # select feature columns
    else:
        df_metrics_all[modality]=df_metrics_all["func_fc"].groupby(["IDs","group","age","sex","labels1","level_labels"])["fcorr"].mean().reset_index()
        df_metrics_all[modality]=df_metrics_all["func_fc"][df_metrics_all["func_fc"]["labels1"].isin(["ventral","dorsal"])].reset_index(drop=True)

#----- merge structural and functional df
df_metrics_VD=df_metrics_all["func_tp"].groupby(["IDs","age","sex","ventro_dorsal","levels"])[list(functional_features)].mean().reset_index()
df_metrics_VD["fc"]=df_metrics_all["func_fc"]["fcorr"]

#--- Reduce the df by one value / participant
df_metrics_VD_reduced=df_metrics_VD.groupby(["ventro_dorsal","levels"])[list(functional_features) + ['fc']].mean().reset_index()

### <font color=#5b797e> B. FC and temporal similarity coupling
#### <font color=#5b797e> B.1 Global effect

In [ ]:
df_fc_temp,data_fc_temp=stat_func.compute_regional_coupling(matrix1=matrix["func_fc"][:, :, :],matrix2=matrix["func_tp"][:, :, :],metrics=["fc","temp"],df_out=True,metadata_df=metadata)

r_value, p_value = spearmanr(data_fc_temp['fc'], data_fc_temp["temp"])
print(r_value); print(p_value)
data_fc_temp["fc_ln"]=np.log(data_fc_temp["fc"])
data_fc_temp["temp_ln"]=np.log(data_fc_temp["temp"])
plot.regplots(df=data_fc_temp,x_data="fc",y_data="temp",reg_color="#5b797e",x_color="#04AF98",y_color="#B14263",
             save=False,output_f=output_dir + "/figures/fc_temp_coupling")


#data_fc_temp.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_2C_SpiFC_SpiDyn_coupling.csv", index=False)

In [ ]:
###### #---- Compute spearman correlation between each temporal and functional features
results_df = {}; significant= {}
results=[]
df=df_metrics_VD_reduced
for s_feat in functional_features:
    for f_feat in ['fc']:
        r, p = spearmanr(df[s_feat], df[f_feat], nan_policy='omit')
        results.append({"functional_feature": s_feat,
                             'fc': f_feat,
                             "spearman_r": r,
                             "p_value": p})
results_df = pd.DataFrame(results)
significant = results_df[results_df["p_value"] < 0.05].sort_values(by="spearman_r", ascending=False)


#---- Plot the results in a heatmap
colors = ["#2C1562","#61508A","#968AB1","#FFFFFF","#FACF80","#F7B740","#F49E00"]  # blue → white → red
custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)

# Pivot matrix for heatmap values (Spearman r)
heatmap_data = results_df.pivot_table(columns='fc',index='functional_feature',values='spearman_r')
annot_data = results_df.pivot_table(columns='fc',index='functional_feature',values='p_value')
annot_stars = annot_data.map(lambda p: '*' if p < 0.05 else '') # Annotate with "*" where p < 0.05, else blank

heatmap_data = heatmap_data.sort_index()
annot_stars = annot_stars.sort_index()
heatmap_data = heatmap_data.sort_values(by='fc', ascending=False)
annot_stars = annot_stars.reindex(index=heatmap_data.index)

# Plot heatmap
plt.figure(figsize=(1, 5))
sns.heatmap(heatmap_data,annot=annot_stars,fmt='s',center=0,linewidths=0.5,cbar_kws={'label': 'Spearman r'},
           cmap=custom_cmap)

plt.title(' Spearman Correlation Between FC and Functional Features\n(* = p < 0.05)')
plt.xlabel('Functional Feature')
plt.ylabel('Dynamic Temp Feature')
plt.tight_layout()
plt.show()


#### <font color=#5b797e> B.2 Age effect

In [ ]:
roi_results_fc=[];roi_results_sim=[]

# compute age effect by roi
for roi in all_df["rois"].unique():
    df_roi_metrics=all_df[all_df["rois"]==roi]
    signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="fc",predictor="age",covariates=["sex"])
    roi_results_fc.append({"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
    signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="sim",predictor="age",covariates=["sex"])
    roi_results_sim.append({"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
    
    roi_results_fc_df= pd.DataFrame(roi_results_fc)
    roi_results_sim_df= pd.DataFrame(roi_results_sim)

# convert the results in a DF
all_age_eff=pd.DataFrame({"rois":roi_results_fc_df["rois"],
            "stat_fc":roi_results_fc_df["tvalue_age"],
            "stat_sim":roi_results_sim_df["tvalue_age"]})

# Compute spearman correlation between tvalues
r_value, p_value = spearmanr(all_age_eff["stat_fc"], all_age_eff["stat_sim"])
print(r_value); print(p_value)

# plot the results
plot.regplots(df=all_age_eff,x_data="stat_fc",y_data="stat_sim",reg_color="#5b797e",x_color="#B14263",y_color="#04AF98"
              ,ymin=-3.5, ymax=3.5,xmin=-3, xmax=3.5, save=False,output_f=output_dir + "/fc_temp_coupling_age")

#all_age_eff.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_2C_SpiFC_SpiDyn_coupling_aging.csv", index=False)